# Chapter 4: Complete Example
In this chapter, we show two complete examples
1. comparing three decision making models using negative log-likelihood on simulated data
2. comparing three associative learning models using Bayesian Information Criterion on simulated data

## Imports

In [1]:
from ldmunit.models import decision_making, associative_learning
from ldmunit.models.utils import multi_from_single_cls
from ldmunit.testing import InteractiveTest
from ldmunit.scores import NLLScore, BICScore
from ldmunit.utils import partialclass

from os.path import join as pathjoin
from os import getcwd
import sciunit
sciunit.settings['CWD'] = getcwd()
DATA_PATH = 'data'

from read_example_data import get_simulation_data, get_model_params

N_SUBJECTS = 3

## Negative Log-likelihood Tests
First we define the test objects. Each test object will test against a different simulated data we read from the filesystem. A couple of things to note:

1. We define the tests to be multi-subject tests, and later use multi-subject models
2. Any type of score can be passed in runtime. Here, we also need to define lower and upper bounds for the score objects.

In [2]:
nll_test_rwck_data = InteractiveTest(
    multi_subject=True,
    name='RWCK NLL Test',
    observation=get_simulation_data(pathjoin(DATA_PATH, 'multi-rwck.csv'), N_SUBJECTS),
    score_type=partialclass(NLLScore, min_score=0, max_score=1e4),
)

nll_test_rr_data = InteractiveTest(
    multi_subject=True,
    name='RR NLL Test',
    observation=get_simulation_data(pathjoin(DATA_PATH, 'multi-random-responding.csv'), N_SUBJECTS),
    score_type=partialclass(NLLScore, min_score=0, max_score=1e4),
)

nll_test_nwsls_data = InteractiveTest(
    multi_subject=True,
    name='NWSLS NLL Test',
    observation=get_simulation_data(pathjoin(DATA_PATH, 'multi-nwsls.csv'), N_SUBJECTS),
    score_type=partialclass(NLLScore, min_score=0, max_score=1e4),
)

## Decision Making Models
Then we define multi-subject models, which can easily be created from single-subject models using a utility function we provide.

In [3]:
MultiRWCKModel = multi_from_single_cls(decision_making.RWCKModel)
MultiNWSLSModel = multi_from_single_cls(decision_making.NWSLSModel)
MultiRandomRespondModel = multi_from_single_cls(decision_making.RandomRespondModel)

n_action = 3
n_obs = 3

multi_rwck = MultiRWCKModel(n_subj=N_SUBJECTS, n_action=n_action, n_obs=n_obs)
multi_rr = MultiRandomRespondModel(n_subj=N_SUBJECTS, n_action=n_action, n_obs=n_obs)
multi_nwsls = MultiNWSLSModel(n_subj=N_SUBJECTS, n_action=n_action, n_obs=n_obs)

## Run Test Suite

In [4]:
nll_suite = sciunit.TestSuite([nll_test_rwck_data, nll_test_rr_data, nll_test_nwsls_data], name="NLL suite")
nll_suite.judge([multi_rwck, multi_rr, multi_nwsls])

12-26-19 12:55 ldmunit      INFO     RWCK NLL Test : Optimizing RWCKModel model...


12-26-19 12:55 ldmunit      INFO     RR NLL Test : Optimizing RWCKModel model...


12-26-19 12:55 ldmunit      INFO     NWSLS NLL Test : Optimizing RWCKModel model...


12-26-19 12:55 ldmunit      INFO     RWCK NLL Test : Optimizing RandomRespondModel model...


12-26-19 12:55 ldmunit      INFO     RR NLL Test : Optimizing RandomRespondModel model...
/home/eozd/anaconda3/envs/ldmunit/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:2808: RuntimeWarning: invalid value encountered in log
  return log(self._pmf(k, *args))


12-26-19 12:55 ldmunit      INFO     NWSLS NLL Test : Optimizing RandomRespondModel model...


12-26-19 12:55 ldmunit      INFO     RWCK NLL Test : Optimizing NWSLSModel model...
/home/eozd/anaconda3/envs/ldmunit/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:2808: RuntimeWarning: divide by zero encountered in log
  return log(self._pmf(k, *args))


12-26-19 12:55 ldmunit      INFO     RR NLL Test : Optimizing NWSLSModel model...


12-26-19 12:55 ldmunit      INFO     NWSLS NLL Test : Optimizing NWSLSModel model...


,RWCK NLL Test,RR NLL Test,NWSLS NLL Test
RWCKModel,113,123,109
RandomRespondModel,117,155,111
NWSLSModel,416,115,87.6


## BIC Tests

In [15]:
def bic_kwargs_fn(model, obs, pred):
    return {"n_model_params": model.n_params(), "n_samples": len(obs["stimuli"])}

bic_test_krw_data = InteractiveTest(
    multi_subject=True,
    name='RWCK BIC Test',
    observation=get_simulation_data(pathjoin(DATA_PATH, 'multi-krw_norm.csv'), N_SUBJECTS, True),
    score_type=partialclass(BICScore, min_score=0, max_score=1e4),
    fn_kwargs_for_score=bic_kwargs_fn
)

bic_test_lsspd_data = InteractiveTest(
    multi_subject=True,
    name='LSSPD BIC Test',
    observation=get_simulation_data(pathjoin(DATA_PATH, 'multi-lsspd.csv'), N_SUBJECTS, True),
    score_type=partialclass(BICScore, min_score=0, max_score=1e4),
    fn_kwargs_for_score=bic_kwargs_fn
)

bic_test_bb_data = InteractiveTest(
    multi_subject=True,
    name='BetaBinomial BIC Test',
    observation=get_simulation_data(pathjoin(DATA_PATH, 'multi-beta_binomial.csv'), N_SUBJECTS, True),
    score_type=partialclass(BICScore, min_score=0, max_score=1e4),
    fn_kwargs_for_score=bic_kwargs_fn
)

## Associative Learning Models

In [16]:
MultiKrwNormModel = multi_from_single_cls(associative_learning.KrwNormModel)
MultiBetaBinomialModel = multi_from_single_cls(associative_learning.BetaBinomialModel)
MultiLSSPDModel = multi_from_single_cls(associative_learning.LSSPDModel)

multi_krw_norm = MultiKrwNormModel(n_subj=N_SUBJECTS, n_obs=4)
multi_lsspd = MultiLSSPDModel(n_subj=N_SUBJECTS, n_obs=4)
multi_bb = MultiBetaBinomialModel(n_subj=N_SUBJECTS, n_obs=4)

## Run Test Suite

In [17]:
bic_al_suite = sciunit.TestSuite([bic_test_krw_data, bic_test_lsspd_data, bic_test_bb_data], name="BIC suite for associative learning")
bic_al_suite.judge([multi_krw_norm, multi_lsspd, multi_bb])

12-26-19 13:02 ldmunit      INFO     RWCK BIC Test : Optimizing KrwNorm model...


12-26-19 13:02 ldmunit      INFO     LSSPD BIC Test : Optimizing KrwNorm model...


12-26-19 13:02 ldmunit      INFO     BetaBinomial BIC Test : Optimizing KrwNorm model...


12-26-19 13:02 ldmunit      INFO     RWCK BIC Test : Optimizing LSSPD model...


12-26-19 13:02 ldmunit      INFO     LSSPD BIC Test : Optimizing LSSPD model...


12-26-19 13:02 ldmunit      INFO     BetaBinomial BIC Test : Optimizing LSSPD model...


12-26-19 13:02 ldmunit      INFO     RWCK BIC Test : Optimizing Beta Binomial model...


12-26-19 13:04 ldmunit      INFO     LSSPD BIC Test : Optimizing Beta Binomial model...


12-26-19 13:05 ldmunit      INFO     BetaBinomial BIC Test : Optimizing Beta Binomial model...


,RWCK BIC Test,LSSPD BIC Test,BetaBinomial BIC Test
KrwNorm,1.87e+03,1.69e+03,1.85e+03
LSSPD,1.68e+03,2.19e+03,979
Beta Binomial,2.84e+03,866,905
